In [1]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import os

In [2]:
root_directory_path = "C:/Users/jayak/Downloads/Python workspace/infosys final work/pythonProject/ty3" 

In [3]:
def list_folders(directory):
    try:
        # Get the list of all files and directories
        items = os.listdir(directory)
        # Filter out the folders
        folders = [item for item in items if os.path.isdir(os.path.join(directory, item))]
        return folders
    except FileNotFoundError:
        return f"The directory '{directory}' does not exist."
    except PermissionError:
        return f"Permission denied to access the directory '{directory}'."
    except Exception as e:
        return f"An error occurred: {e}"

# Example usage
exercise = list_folders(root_directory_path)
print("Exercise in the directory:", exercise)

Exercise in the directory: ['deadlift', 'hammer curl', 'pull Up', 'push-up', 'squat']


In [4]:
def list_all_files(root_directory):
    all_files = []

    for dirpath, dirnames, filenames in os.walk(root_directory):
        for file in filenames:
            full_path = os.path.join(dirpath, file)
            all_files.append(full_path)
    
    return all_files


In [5]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
angle_pairs = [
            ('LEFT_SHOULDER', 'LEFT_ELBOW', 'LEFT_WRIST'),
            ('RIGHT_SHOULDER', 'RIGHT_ELBOW', 'RIGHT_WRIST'),
            ('LEFT_HIP', 'LEFT_KNEE', 'LEFT_ANKLE'),
            ('RIGHT_HIP', 'RIGHT_KNEE', 'RIGHT_ANKLE'),
            ('LEFT_SHOULDER', 'LEFT_HIP', 'LEFT_KNEE'),
            ('RIGHT_SHOULDER', 'RIGHT_HIP', 'RIGHT_KNEE'),
            ('LEFT_ELBOW', 'LEFT_SHOULDER', 'LEFT_HIP'),
            ('RIGHT_ELBOW', 'RIGHT_SHOULDER', 'RIGHT_HIP')
            # Add more pairs as needed
        ]


"""video_path = 'C:\\Users\\jayak\\Downloads\\Python workspace\\infosys\\ty\\push-up\\push-up_41.mp4'"""

"video_path = 'C:\\Users\\jayak\\Downloads\\Python workspace\\infosys\\ty\\push-up\\push-up_41.mp4'"

In [6]:
def calculate_angle(a, b, c):
    a = np.array(a)  
    b = np.array(b)  
    c = np.array(c)  

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

In [7]:
def create_csv(all_files,label):
    angles_data = []
    for video_path in all_files:   
        cap = cv2.VideoCapture(video_path)

        frame_count = 0  # Initialize frame_count here

        interval = 1  # Interval to process frames

        # Define pairs of landmarks to calculate angles
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1
            if frame_count % interval == 0:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                result = pose.process(frame_rgb)

                if result.pose_landmarks:
                    landmarks = result.pose_landmarks.landmark

                    # Collect angles for each specified pair
                    frame_angles = {'Frame': frame_count}
                    for pair in angle_pairs+[label]:
                        if pair not in exercise:
                            a = [landmarks[getattr(mp_pose.PoseLandmark, pair[0]).value].x,
                            landmarks[getattr(mp_pose.PoseLandmark, pair[0]).value].y]
                            b = [landmarks[getattr(mp_pose.PoseLandmark, pair[1]).value].x,
                            landmarks[getattr(mp_pose.PoseLandmark, pair[1]).value].y]
                            c = [landmarks[getattr(mp_pose.PoseLandmark, pair[2]).value].x,
                            landmarks[getattr(mp_pose.PoseLandmark, pair[2]).value].y]

                            angle = calculate_angle(a, b, c)
                            frame_angles[f'{pair[0]}-{pair[1]}-{pair[2]}'] = angle
                        else:
                            frame_angles["Exercise"]=pair

                    # Append the frame's angles to the list
                    angles_data.append(frame_angles)

        cap.release()

        # Save angles data to a CSV file for each video
    df = pd.DataFrame(angles_data)
    df.to_csv(f"{label}.csv", index=False)

In [9]:
for label in exercise:
    all_files=list_all_files(root_directory_path+"/"+label)
    create_csv(all_files,label)

    

In [10]:
def merge_csv(directory):
    df_list = []
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            df = os.path.join(directory, filename)
            df_list.append(df)
    combine_df= pd.read_csv(df_list[0])
    for i in range(1,len(df_list)):
        df=pd.read_csv(df_list[i])
        combine_df=pd.concat([combine_df,df], ignore_index=True)
    
    combine_df.to_csv('jayakrishna_combine_angles.csv', index=False) 
base_path = "C:/Users/jayak/Downloads/Python workspace/infosys final work/pythonProject" 
merge_csv(base_path)